# 0. Se cargan las librerías de trabajo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Se cargan los datos desde el drive.

In [ ]:
ruta = "/content/drive/MyDrive/Analítica de Negocios/2. BD2_In_App_Pur_Int.xlsx"
data = pd.read_excel(ruta, sheet_name = 0)
data = data.dropna()
data.head()

# Seleccionamos las variables de interés.
datos = data[["Age", "Income", "Years", "Hours"]]
datos.head()
datos = np.array(datos)
yd = data[["Buy"]]; yd = np.array(yd)

# 2. Se procede con la construcción del modelo del Árbol.

In [ ]:
mar = DecisionTreeClassifier(criterion = "gini", max_depth = 4)
mar.fit(datos,yd)
ydp = mar.predict(datos)

# Evaluamos el modelo
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(yd,ydp)
VN = cm[0,0]; FP = cm[0,1]; FN = cm[1,0]; VP = cm[1,1]

Ex = (VP + VN)/(VP + VN + FP + FN)
Sen = VP/(VP + FN); Esp = VN/(VN + FP)

print(cm)
print("Exactitud: ", Ex)
print("Sensibilidad: ", Sen)
print("Especificidad: ", Esp)

[[139  50]
 [ 11 200]]
Exactitud:  0.8475
Sensibilidad:  0.9478672985781991
Especificidad:  0.7354497354497355


# 3. Se procede con la visualización del árbol.

In [ ]:
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

Va = ["Age", "Income", "Years", "Hours"]

dot_data = export_graphviz(mar, feature_names = Va)
graph = graph_from_dot_data(dot_data)
graph.write_png("Árbol - Reto.png")

True

# 4. Le calculamos los valores de preaprobación a los solicitantes nuevos.

In [ ]:
data2 = pd.read_excel(ruta, sheet_name = 1)
data2 = data2[["Age", "Income", "Years", "Hours"]]
data2 = data2.dropna()

datos2 = np.array(data2)
ydp2 = mar.predict(datos2)
data2["Buy"] = ydp2
data2.head()

data2.to_excel("NuevosCompradores.xlsx")

# Análisis de resultados.

En la base de datos se puede observar un total de 400 jugadores, de los cuales 211 jugadores realizarían compras en la aplicación (52.75%) y 189 jugadores no (47.25%). En base a la información anterior, se realiza un modelo de árbol de decisión con las variables edad (expresada en años), ingresos (que es la ganancia de cada jugador), años que ha jugado y las horas que le dedica al juego. Este modelo presenta un índice de exactitud del 84.75%, lo que indica que ese es el porcentaje de datos clasificados correctamente y que se puede considerar un modelo aceptable para el pronóstico de los datos. Además, este modelo presenta una sensibilidad del 94.79% y una especificidad del 73.54%, indicando el comportamiento del modelo al clasificar a los jugadores que comprarían y los que no, respectivamente; y demostrando que predomina la clasificación de jugadores que comprarían.

En el árbol podemos observar 3 nodos puros (gini = 0): El primero cumple con la regla de tener un ingreso menor a 63.5, esto lleva a que de los 90 jugadores que hay en este nodo, el 100% no compraría en el juego. El segundo nodo cumple con las reglas de tener un ingreso mayor a 63.5, tener menos de 1.5 años jugando y una edad mayor a 23, donde el 100% de los jugadores en este nodo (28) no comprarían en el juego. Y el último nodo cumple con la regla de un ingreso mayor a 63.5, tener menos de 1.5 años jugando, una edad menor a 23 y llevar más de 0.5 años jugando, mostrando que 5 de los jugadores de este nodo no comprarían en el juego, es decir el 100%.

De los dos nuevos jugadores, el primero cumple con las reglas de tener un ingreso mayor a 63.5, tener más de 1.5 años jugando, tener menos de 5.5 años jugando y un ingreso mayor a 76.5, por lo cual tiene un 66.96% de que sí realizaría una compra en el juego. Mientras que el segundo jugador cumple con las reglas de tener un ingreso mayor a 63.5, tener menos de 1.5 años jugando y una edad mayor a 23, entrando en el segundo nodo puro anteriormente mencionado, por lo cual tiene un 100% de probabilidad de no realizar una compra en el juego.

Una persona que cumpla las siguientes reglas Income>63.5 AND Years>1.5 AND Years>5.5 AND Income<=137.5 tendría una probabilidad del 91.67% de probabilidad de realizar una compra en el juego.